In [ ]:
!pip install webrtcvad
!pip install spectralcluster
!pip install ffmpeg-python
!pip install Speechrecognition
!pip install pydub
!pip install azure-cognitiveservices-speech

In [ ]:
from pydub import AudioSegment
def mp3_to_wav(audio_file_path):
    sound = AudioSegment.from_mp3(audio_file_path)
    audio_file_path = audio_file_path.split('.')[0] + '.wav'
    sound.export(audio_file_path, format="wav")
    return audio_file_path

audio_file_path = mp3_to_wav('/content/sample1audio.wav')
print(audio_file_path)

In [ ]:
!git clone https://github.com/resemble-ai/Resemblyzer.git
%cd Resemblyzer

In [ ]:
#pip install webrtcvad
from resemblyzer import preprocess_wav, VoiceEncoder
from pathlib import Path

#give the file path to your audio file
audio_file_path = '/content/sample1audio.wav'
wav_fpath = Path(audio_file_path)

wav = preprocess_wav(wav_fpath)
encoder = VoiceEncoder("cpu")
_, cont_embeds, wav_splits = encoder.embed_utterance(wav, return_partials=True, rate=16)
print(cont_embeds.shape)

In [ ]:
from spectralcluster import SpectralClusterer
from spectralcluster import RefinementOptions
from spectralcluster import ThresholdType
from spectralcluster import ICASSP2018_REFINEMENT_SEQUENCE

clusterer = SpectralClusterer(
    min_clusters=2,
    max_clusters=100,
)
refinement_options = RefinementOptions(
    gaussian_blur_sigma=1,
    p_percentile=0.90)
labels = clusterer.predict(cont_embeds)

In [ ]:
def create_labelling(labels,wav_splits):
    from resemblyzer import sampling_rate
    times = [((s.start + s.stop) / 2) / sampling_rate for s in wav_splits]
    labelling = []
    start_time = 0
    
    for i,time in enumerate(times):
        if i>0 and labels[i]!=labels[i-1]:
            temp = [str(labels[i-1]),start_time,time]
            labelling.append(tuple(temp))
            start_time = time
        if i==len(times)-1:
            temp = [str(labels[i]),start_time,time]
            labelling.append(tuple(temp))
    return labelling
  
labelling = create_labelling(labels,wav_splits)

In [ ]:
labelling

In [ ]:
%cd ..

In [ ]:
import ffmpeg
import speech_recognition as sr
import time
import os
a=0
r = sr.Recognizer()
for label in labelling:
  #print('1')
  a=a+1
  speaker=label[0]
  start=label[1]
  end=label[2]
  audio_input = ffmpeg.input('/content/Driving-English-Conversation-Sample.wav')
  #print(start)
  #print(end)
  audio_cut = audio_input.audio.filter('atrim', start=start,end=end)
  audio_output = ffmpeg.output(audio_cut, f'{a}out.wav')
  #print('2')
  try:
    #print('3')
    ffmpeg.run(audio_output)
  except:
    #print('4')
    pass
  #time.sleep()
  with sr.AudioFile(f'{a}out.wav') as source:
    #print('5')
    audio_text = r.listen(source)
# recoginize_() method will throw a request error if the API is unreachable, hence using exception handling
    try:
      #print('6')
        # using google speech recognition
      text = r.recognize_google(audio_text)
      print('Converting audio transcripts into text ...')
      #print('7')
      print(f'{speaker} : {text}')
    except:
      #print('8')
      print('Sorry.. run again...')
    time.sleep(1)
    
  os.remove(f'{a}out.wav')


In [ ]:
import ffmpeg
import azure.cognitiveservices.speech as speechsdk
import speech_recognition as sr
import time
import os
a=0
speech_key, service_region = "speech_key", "service_region"
speech_config=speechsdk.SpeechConfig(subscription=speech_key, region=service_region)

# Creates an audio configuration that points to an audio file.
# Replace with your own audio filename.
audio_filename = "/content/sample1audio.wav"
audio_input = speechsdk.audio.AudioConfig(filename=audio_filename)

# Creates a recognizer with the given settings
speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_input)

#print("Recognizing first result...")

for label in labelling:
  #print('1')
  a=a+1
  speaker=label[0]
  start=label[1]
  end=label[2]
  audio_input = ffmpeg.input('/content/sample1audio.wav')
  #print(start)
  #print(end)
  audio_cut = audio_input.audio.filter('atrim', start=start,end=end)
  audio_output = ffmpeg.output(audio_cut, f'{a}out.wav')
  #print('2')
  try:
    #print('3')
    ffmpeg.run(audio_output)
  except:
    #print('4')
    pass
  #time.sleep()
  with sr.AudioFile(f'{a}out.wav') as source:
    result = speech_recognizer.recognize_once()

    # Checks result.
    if result.reason == speechsdk.ResultReason.RecognizedSpeech:
        #print("Recognized by Azure: {}".format(result.text))
        #print(speaker)
        #'''
        if speaker=='0':
          print(f"Agent: {result.text}")
        else:
          print(f"Customer: {result.text}")
          #'''
        #print(f"{speaker}: {result.text}")
    elif result.reason == speechsdk.ResultReason.NoMatch:
        print("No speech could be recognized: {}".format(result.no_match_details))
        #pass
    elif result.reason == speechsdk.ResultReason.Canceled:
        cancellation_details = result.cancellation_details
        #print("Speech Recognition canceled: {}".format(cancellation_details.reason))
        pass
        if cancellation_details.reason == speechsdk.CancellationReason.Error:
            print("Error details: {}".format(cancellation_details.error_details))


  os.remove(f'{a}out.wav')